In [ ]:
#json file name: amazon_Women_dataset

In [1]:
import pandas as pd
import json
import re
import string
from textblob import Word
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.stem import PorterStemmer
porter = PorterStemmer()
import demoji


In [2]:

#step 1: load data from json into a dataframe

file_path = 'amazondata.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
df = pd.json_normalize(data)

In [3]:
df.head()

,productUrl,productId,gender,manufacturer,category,description,subCategory,price,productName,brandName,rating,product reviews,images.src,colors and images.src,colors and images.alt
0,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,ASIN : B0C843K9P9,Department : mens,Manufacturer : Amazon Essentials,men fashion,SLIM FIT: Slim fit through the hip and thigh w...,"shorts , tshirts , jackets",$12,"Amazon Essentials Men's Slim-Fit 9"" Short",Amazon Essentials Store,4.4,"[{'r_name': 'Amazon Customer', 'date_r': 'Revi...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31B35LtII...,"[Black, Blue, Grey, Grey Pineapple, Grey Plaid..."
1,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,ASIN : B0CQRJVB5F,Department : mens,Manufacturer : Wrangler Authentics Men's Sport...,men fashion,RELAXED FIT. These cargo shorts sit at the nat...,"shorts , tshirts , jackets",$26,Wrangler Authentics Men's Premium Twill Cargo ...,Wrangler Authentics Store,4.5,"[{'r_name': 'Devin', 'date_r': 'Reviewed in th...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/41kcIogGe...,"[Acorn Twill, Bullfrog, Camel, Forest Green Ca..."
2,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,ASIN : B09T45VHWX,Department : mens,Manufacturer : Fruit of the Loom,men fashion,"Relaxed Fit, 2 Pack Jersey Shorts - 20 plus UP...","shorts , tshirts , jackets",$19,Fruit of the Loom Men's Eversoft Cotton Shorts...,Fruit of the Loom Store,4.5,"[{'r_name': 'MarkE', 'date_r': 'Reviewed in th...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/21UMhFwFu...,"[2 Pack - Black, 2 Pack - Black Heather, 2 Pac..."
3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,ASIN : B0BW1YJ38R,Department : mens,Manufacturer : Lee,men fashion,STRAIGHT FIT. With a straight fit from the sea...,"shorts , tshirts , jackets",$24,Lee Men's Extreme Motion Flat Front Short,Lee Store,4.6,"[{'r_name': 'Richard Dav', 'date_r': 'Reviewed...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31js-qcsQ...,"[Black, Blue Mirage, Charcoal, Clay Rose, Iron..."
4,https://www.amazon.com/Wrangler-Authentics-Cla...,ASIN : B08RBHL3HW,Department : mens,Manufacturer : Wrangler,men fashion,TRADITIONAL CARGO STYLE. Built for versatility...,"shorts , tshirts , jackets",$26.85,Wrangler Authentics Men's Classic Cargo Stretc...,Wrangler Authentics Store,4.5,"[{'r_name': 'Founding Father', 'date_r': 'Revi...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31k1p6cNC...,"[Anthracite, Anthracite Camo, Black Twill, Cop..."


In [4]:
#step 2: rename column names into easy names
df.rename(columns={'productId': 'ASIN'}, inplace=True)
df.rename(columns={'productName': 'title'}, inplace=True)
df.rename(columns={'colors and images.alt': 'colors'}, inplace=True)
df.rename(columns={'colors and images.src': 'color_imgs'}, inplace=True)
df.rename(columns={'images.src': 'images'}, inplace=True)
df.rename(columns={'productUrl': 'url'}, inplace=True)
df.head()


,url,ASIN,gender,manufacturer,category,description,subCategory,price,title,brandName,rating,product reviews,images,color_imgs,colors
0,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,ASIN : B0C843K9P9,Department : mens,Manufacturer : Amazon Essentials,men fashion,SLIM FIT: Slim fit through the hip and thigh w...,"shorts , tshirts , jackets",$12,"Amazon Essentials Men's Slim-Fit 9"" Short",Amazon Essentials Store,4.4,"[{'r_name': 'Amazon Customer', 'date_r': 'Revi...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31B35LtII...,"[Black, Blue, Grey, Grey Pineapple, Grey Plaid..."
1,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,ASIN : B0CQRJVB5F,Department : mens,Manufacturer : Wrangler Authentics Men's Sport...,men fashion,RELAXED FIT. These cargo shorts sit at the nat...,"shorts , tshirts , jackets",$26,Wrangler Authentics Men's Premium Twill Cargo ...,Wrangler Authentics Store,4.5,"[{'r_name': 'Devin', 'date_r': 'Reviewed in th...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/41kcIogGe...,"[Acorn Twill, Bullfrog, Camel, Forest Green Ca..."
2,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,ASIN : B09T45VHWX,Department : mens,Manufacturer : Fruit of the Loom,men fashion,"Relaxed Fit, 2 Pack Jersey Shorts - 20 plus UP...","shorts , tshirts , jackets",$19,Fruit of the Loom Men's Eversoft Cotton Shorts...,Fruit of the Loom Store,4.5,"[{'r_name': 'MarkE', 'date_r': 'Reviewed in th...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/21UMhFwFu...,"[2 Pack - Black, 2 Pack - Black Heather, 2 Pac..."
3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,ASIN : B0BW1YJ38R,Department : mens,Manufacturer : Lee,men fashion,STRAIGHT FIT. With a straight fit from the sea...,"shorts , tshirts , jackets",$24,Lee Men's Extreme Motion Flat Front Short,Lee Store,4.6,"[{'r_name': 'Richard Dav', 'date_r': 'Reviewed...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31js-qcsQ...,"[Black, Blue Mirage, Charcoal, Clay Rose, Iron..."
4,https://www.amazon.com/Wrangler-Authentics-Cla...,ASIN : B08RBHL3HW,Department : mens,Manufacturer : Wrangler,men fashion,TRADITIONAL CARGO STYLE. Built for versatility...,"shorts , tshirts , jackets",$26.85,Wrangler Authentics Men's Classic Cargo Stretc...,Wrangler Authentics Store,4.5,"[{'r_name': 'Founding Father', 'date_r': 'Revi...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31k1p6cNC...,"[Anthracite, Anthracite Camo, Black Twill, Cop..."


In [5]:
#step 3: edit the data accordingly.

#removing ASIN , Department and manufacturer from the rows:
df['ASIN'] = df['ASIN'].str.extract('ASIN : (\S+)', expand=False)
df['gender'] = df['gender'].str.extract('Department : (\S+)', expand=False)
df['manufacturer'] = df['manufacturer'].str.extract('Manufacturer : (.+)', expand=False)
df['price'] = df['price'].str.replace('$', '').astype(float)
df.head()


C:\Users\Khadeeja Shah\AppData\Local\Temp\ipykernel_12952\131263546.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] = df['price'].str.replace('$', '').astype(float)


,url,ASIN,gender,manufacturer,category,description,subCategory,price,title,brandName,rating,product reviews,images,color_imgs,colors
0,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B0C843K9P9,mens,Amazon Essentials,men fashion,SLIM FIT: Slim fit through the hip and thigh w...,"shorts , tshirts , jackets",12.00,"Amazon Essentials Men's Slim-Fit 9"" Short",Amazon Essentials Store,4.4,"[{'r_name': 'Amazon Customer', 'date_r': 'Revi...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31B35LtII...,"[Black, Blue, Grey, Grey Pineapple, Grey Plaid..."
1,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B0CQRJVB5F,mens,Wrangler Authentics Men's Sportswear,men fashion,RELAXED FIT. These cargo shorts sit at the nat...,"shorts , tshirts , jackets",26.00,Wrangler Authentics Men's Premium Twill Cargo ...,Wrangler Authentics Store,4.5,"[{'r_name': 'Devin', 'date_r': 'Reviewed in th...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/41kcIogGe...,"[Acorn Twill, Bullfrog, Camel, Forest Green Ca..."
2,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B09T45VHWX,mens,Fruit of the Loom,men fashion,"Relaxed Fit, 2 Pack Jersey Shorts - 20 plus UP...","shorts , tshirts , jackets",19.00,Fruit of the Loom Men's Eversoft Cotton Shorts...,Fruit of the Loom Store,4.5,"[{'r_name': 'MarkE', 'date_r': 'Reviewed in th...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/21UMhFwFu...,"[2 Pack - Black, 2 Pack - Black Heather, 2 Pac..."
3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B0BW1YJ38R,mens,Lee,men fashion,STRAIGHT FIT. With a straight fit from the sea...,"shorts , tshirts , jackets",24.00,Lee Men's Extreme Motion Flat Front Short,Lee Store,4.6,"[{'r_name': 'Richard Dav', 'date_r': 'Reviewed...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31js-qcsQ...,"[Black, Blue Mirage, Charcoal, Clay Rose, Iron..."
4,https://www.amazon.com/Wrangler-Authentics-Cla...,B08RBHL3HW,mens,Wrangler,men fashion,TRADITIONAL CARGO STYLE. Built for versatility...,"shorts , tshirts , jackets",26.85,Wrangler Authentics Men's Classic Cargo Stretc...,Wrangler Authentics Store,4.5,"[{'r_name': 'Founding Father', 'date_r': 'Revi...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31k1p6cNC...,"[Anthracite, Anthracite Camo, Black Twill, Cop..."


In [6]:
df['product reviews'].loc[0]

[{'r_name': 'Amazon Customer',
  'date_r': 'Reviewed in the United States on December 14, 2023',
  'review': 'I am 5\'7"-5\'8", about 185 lbs. These shorts fit me just right, but if I were to gain 5-10 extra lbs, I would have to go up in waist size.\n\nThese shorts have an excellent "fit." The length is perfect for me. They fit me above the knee, but not too high up where they look like short-shorts. The pockets are deep enough to help keep your items from falling out. Also, I notice they have a YKK brand zipper, which are sturdy, and don\'t de-rail. I don\'t see them much anymore, so I was pleasantly surprised with that quality of craftsmanship. Finally, these shorts have a nice, simple label with the waist size, and another one with the style "e.g., slim" which I find helpful in case I was to re-buy this or a similar product.'},
 {'r_name': 'Max',
  'date_r': 'Reviewed in the United States on January 5, 2024',
  'review': 'It’s okay, Nothing fancy, comforting to wear, inexpensive , w

In [7]:
#fixing reviews column
df['reviewer_name'] = df['product reviews'].apply(lambda reviews: ', '.join(review['r_name'] for review in reviews))
df['review_date'] = df['product reviews'].apply(lambda reviews: ', '.join(review['date_r'].replace('Reviewed in the United States on ', '') for review in reviews))
df['review'] = df['product reviews'].apply(lambda reviews: ', '.join(review['review'] for review in reviews))

df.head()



,url,ASIN,gender,manufacturer,category,description,subCategory,price,title,brandName,rating,product reviews,images,color_imgs,colors,reviewer_name,review_date,review
0,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B0C843K9P9,mens,Amazon Essentials,men fashion,SLIM FIT: Slim fit through the hip and thigh w...,"shorts , tshirts , jackets",12.00,"Amazon Essentials Men's Slim-Fit 9"" Short",Amazon Essentials Store,4.4,"[{'r_name': 'Amazon Customer', 'date_r': 'Revi...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31B35LtII...,"[Black, Blue, Grey, Grey Pineapple, Grey Plaid...","Amazon Customer, Max, heidi, Lucas Chau, Rober...","December 14, 2023, January 5, 2024, December 3...","I am 5'7""-5'8"", about 185 lbs. These shorts fi..."
1,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B0CQRJVB5F,mens,Wrangler Authentics Men's Sportswear,men fashion,RELAXED FIT. These cargo shorts sit at the nat...,"shorts , tshirts , jackets",26.00,Wrangler Authentics Men's Premium Twill Cargo ...,Wrangler Authentics Store,4.5,"[{'r_name': 'Devin', 'date_r': 'Reviewed in th...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/41kcIogGe...,"[Acorn Twill, Bullfrog, Camel, Forest Green Ca...","Devin, Amazon Customer, Seth A. Tucker, Frankl...","May 29, 2023, February 24, 2024, March 3, 2024...",Comfortable Fit:\nThe cargo shorts feature a r...
2,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B09T45VHWX,mens,Fruit of the Loom,men fashion,"Relaxed Fit, 2 Pack Jersey Shorts - 20 plus UP...","shorts , tshirts , jackets",19.00,Fruit of the Loom Men's Eversoft Cotton Shorts...,Fruit of the Loom Store,4.5,"[{'r_name': 'MarkE', 'date_r': 'Reviewed in th...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/21UMhFwFu...,"[2 Pack - Black, 2 Pack - Black Heather, 2 Pac...","MarkE, Kool Aid, Jan, Sheila, Q, Lu Ann, Micha...","February 26, 2024, March 2, 2024, November 12,...","- Fit as expected\n- Comfortable, good for exe..."
3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B0BW1YJ38R,mens,Lee,men fashion,STRAIGHT FIT. With a straight fit from the sea...,"shorts , tshirts , jackets",24.00,Lee Men's Extreme Motion Flat Front Short,Lee Store,4.6,"[{'r_name': 'Richard Dav', 'date_r': 'Reviewed...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31js-qcsQ...,"[Black, Blue Mirage, Charcoal, Clay Rose, Iron...","Richard Dav, Ibuystuffonline, B. Bad, MMN, BWC...","January 11, 2024, October 1, 2023, January 29,...",I love these shorts. I've just bought my 4th p...
4,https://www.amazon.com/Wrangler-Authentics-Cla...,B08RBHL3HW,mens,Wrangler,men fashion,TRADITIONAL CARGO STYLE. Built for versatility...,"shorts , tshirts , jackets",26.85,Wrangler Authentics Men's Classic Cargo Stretc...,Wrangler Authentics Store,4.5,"[{'r_name': 'Founding Father', 'date_r': 'Revi...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31k1p6cNC...,"[Anthracite, Anthracite Camo, Black Twill, Cop...","Founding Father, John Sparks, Bob H, Amazon Cu...","February 27, 2024, February 16, 2024, November...",They run a bit loose. I'm a 34/36 so I ordered...


In [8]:
# step 4: fix the null values
df.isnull().sum()

url                  0
ASIN                42
gender              44
manufacturer       108
category             0
description          0
subCategory          0
price                0
title                0
brandName            0
rating               0
product reviews      0
images               0
color_imgs           0
colors               0
reviewer_name        0
review_date          0
review               0
dtype: int64

In [9]:

df['manufacturer'] = df['manufacturer'].fillna('AMAZON')
df['gender'] = df['gender'].fillna('women')
fill_values = ['ABCDX' + str(i).zfill(3) for i in range(len(df))]
df['ASIN'] = df['ASIN'].fillna(pd.Series(fill_values))


In [10]:
df.isnull().sum()

url                0
ASIN               0
gender             0
manufacturer       0
category           0
description        0
subCategory        0
price              0
title              0
brandName          0
rating             0
product reviews    0
images             0
color_imgs         0
colors             0
reviewer_name      0
review_date        0
review             0
dtype: int64

In [11]:
df['review'].loc[0]	#reviews_info

'I am 5\'7"-5\'8", about 185 lbs. These shorts fit me just right, but if I were to gain 5-10 extra lbs, I would have to go up in waist size.\n\nThese shorts have an excellent "fit." The length is perfect for me. They fit me above the knee, but not too high up where they look like short-shorts. The pockets are deep enough to help keep your items from falling out. Also, I notice they have a YKK brand zipper, which are sturdy, and don\'t de-rail. I don\'t see them much anymore, so I was pleasantly surprised with that quality of craftsmanship. Finally, these shorts have a nice, simple label with the waist size, and another one with the style "e.g., slim" which I find helpful in case I was to re-buy this or a similar product., It’s okay, Nothing fancy, comforting to wear, inexpensive , wearing this for few days inside my home ,Fits well for me\nSo far I like it ., Runs a couple sizes too small. So beware of that. Good looking shorts but make sure to size up if you’ve got that sexy dad bod.,

In [15]:
#step 4: to make ASIN primary key in database , its imp to have no duplicates. so, removing duplicates.
df.duplicated(subset='ASIN', keep=False).sum()


0

In [14]:
df = df[~df.duplicated(subset='ASIN', keep='first')]
print("DataFrame without duplicates:")
print("now duplicates" ,df.duplicated(subset='ASIN', keep=False).sum())
print(df)


DataFrame without duplicates:
now duplicates 0
                                                    url        ASIN  gender  \
0     https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B0C843K9P9    mens   
1     https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B0CQRJVB5F    mens   
2     https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B09T45VHWX    mens   
3     https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B0BW1YJ38R    mens   
4     https://www.amazon.com/Wrangler-Authentics-Cla...  B08RBHL3HW    mens   
...                                                 ...         ...     ...   
1428  https://www.amazon.com/Avenue-Plus-Size-Trouse...  B0CNWRYVHZ  womens   
1429  https://www.amazon.com/Gold-Toe-Womens-Diamond...  B0CV817MB8  womens   
1430  https://www.amazon.com/Berkshire-womens-Trouse...  B0CP8QJGBW  womens   
1431  https://www.amazon.com/Jostar-Womens-Stretchy-...  B00DOHRUE0   women   
1432  https://www.amazon.com/Amazon-Essentials-Women...  B08JDRD3DJ   women   

    

In [ ]:
#Use natural language processing (NLP) techniques to clean and standardize reviews. This may
#include removing stop words, stemming, and lemmatization. (use NLTK)

In [16]:
# to be done on description and review
#first on description: 
#step 1: special character + punctuation + lowercase


def spl(text):
    cleaned_text = re.sub(r'[^\w\s,\.]', '', text) #punctuation + special characters
    anss = cleaned_text.translate(str.maketrans('', '', string.punctuation))
    ans=anss.lower()
    return ans 

df['decs']=df['description'].apply(spl)
#print(df.head())

#step2: stopwords

english_stopwords = set(stopwords.words("english"))
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in english_stopwords]
    return " ".join(filtered_words)

df['decs'] = df['decs'].apply(remove_stopwords)
#print(df.head())

#step3: lemmatization

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [Word(word).lemmatize("v") for word in words]
    return ' '.join(lemmatized_words)

df["decs"]=df["decs"].apply(lemmatize_text)
print(df.head())

#step 4: stemming

def stem_text(text):
    return porter.stem(text)

df["decs"] = df["decs"].apply(lambda x: stem_text(x))
df.head()


                                                 url        ASIN gender  \
0  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B0C843K9P9   mens   
1  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B0CQRJVB5F   mens   
2  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B09T45VHWX   mens   
3  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B0BW1YJ38R   mens   
4  https://www.amazon.com/Wrangler-Authentics-Cla...  B08RBHL3HW   mens   

                           manufacturer     category  \
0                     Amazon Essentials  men fashion   
1  Wrangler Authentics Men's Sportswear  men fashion   
2                     Fruit of the Loom  men fashion   
3                                   Lee  men fashion   
4                              Wrangler  men fashion   

                                         description  \
0  SLIM FIT: Slim fit through the hip and thigh w...   
1  RELAXED FIT. These cargo shorts sit at the nat...   
2  Relaxed Fit, 2 Pack Jersey Shorts - 20 pl

,url,ASIN,gender,manufacturer,category,description,subCategory,price,title,brandName,rating,product reviews,images,color_imgs,colors,reviewer_name,review_date,review,decs
0,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B0C843K9P9,mens,Amazon Essentials,men fashion,SLIM FIT: Slim fit through the hip and thigh w...,"shorts , tshirts , jackets",12.00,"Amazon Essentials Men's Slim-Fit 9"" Short",Amazon Essentials Store,4.4,"[{'r_name': 'Amazon Customer', 'date_r': 'Revi...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31B35LtII...,"[Black, Blue, Grey, Grey Pineapple, Grey Plaid...","Amazon Customer, Max, heidi, Lucas Chau, Rober...","December 14, 2023, January 5, 2024, December 3...","I am 5'7""-5'8"", about 185 lbs. These shorts fi...",slim fit slim fit hip thigh taper leg open sit...
1,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B0CQRJVB5F,mens,Wrangler Authentics Men's Sportswear,men fashion,RELAXED FIT. These cargo shorts sit at the nat...,"shorts , tshirts , jackets",26.00,Wrangler Authentics Men's Premium Twill Cargo ...,Wrangler Authentics Store,4.5,"[{'r_name': 'Devin', 'date_r': 'Reviewed in th...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/41kcIogGe...,"[Acorn Twill, Bullfrog, Camel, Forest Green Ca...","Devin, Amazon Customer, Seth A. Tucker, Frankl...","May 29, 2023, February 24, 2024, March 3, 2024...",Comfortable Fit:\nThe cargo shorts feature a r...,relax fit cargo short sit natural waist relax ...
2,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B09T45VHWX,mens,Fruit of the Loom,men fashion,"Relaxed Fit, 2 Pack Jersey Shorts - 20 plus UP...","shorts , tshirts , jackets",19.00,Fruit of the Loom Men's Eversoft Cotton Shorts...,Fruit of the Loom Store,4.5,"[{'r_name': 'MarkE', 'date_r': 'Reviewed in th...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/21UMhFwFu...,"[2 Pack - Black, 2 Pack - Black Heather, 2 Pac...","MarkE, Kool Aid, Jan, Sheila, Q, Lu Ann, Micha...","February 26, 2024, March 2, 2024, November 12,...","- Fit as expected\n- Comfortable, good for exe...",relax fit 2 pack jersey short 20 plus upf prot...
3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B0BW1YJ38R,mens,Lee,men fashion,STRAIGHT FIT. With a straight fit from the sea...,"shorts , tshirts , jackets",24.00,Lee Men's Extreme Motion Flat Front Short,Lee Store,4.6,"[{'r_name': 'Richard Dav', 'date_r': 'Reviewed...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31js-qcsQ...,"[Black, Blue Mirage, Charcoal, Clay Rose, Iron...","Richard Dav, Ibuystuffonline, B. Bad, MMN, BWC...","January 11, 2024, October 1, 2023, January 29,...",I love these shorts. I've just bought my 4th p...,straight fit straight fit seat thigh khaki sho...
4,https://www.amazon.com/Wrangler-Authentics-Cla...,B08RBHL3HW,mens,Wrangler,men fashion,TRADITIONAL CARGO STYLE. Built for versatility...,"shorts , tshirts , jackets",26.85,Wrangler Authentics Men's Classic Cargo Stretc...,Wrangler Authentics Store,4.5,"[{'r_name': 'Founding Father', 'date_r': 'Revi...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31k1p6cNC...,"[Anthracite, Anthracite Camo, Black Twill, Cop...","Founding Father, John Sparks, Bob H, Amazon Cu...","February 27, 2024, February 16, 2024, November...",They run a bit loose. I'm a 34/36 so I ordered...,traditional cargo style build versatility comf...


In [17]:
df['description'].loc[0]

'SLIM FIT: Slim fit through the hip and thigh with a tapered leg opening. Sits at the waist. - COTTON TWILL: Gently structured cotton twill that is naturally breathable with a garment wash for a soft, durable hand. - EVERYDAY SHORTS: Classic khaki chino shorts in a non-stretch, garment-washed fabric for a versatile warm-weather wardrobe staple. - DETAILS: Zip fly with button at waist, front slant pockets, and button-through back welt pockets. -  - '

In [18]:

df['decs'].loc[0]

'slim fit slim fit hip thigh taper leg open sit waist cotton twill gently structure cotton twill naturally breathable garment wash soft durable hand everyday short classic khaki chino short nonstretch garmentwashed fabric versatile warmweather wardrobe staple detail zip fly button waist front slant pocket buttonthrough back welt pocket'

In [19]:
# to be done on description and review
#nnow on review: 
#step 1: special character + punctuation + lowercase

def spl(text):
    cleaned_text = re.sub(r'[^\w\s,\.]', '', text) #punctuation + special characters
    anss = cleaned_text.translate(str.maketrans('', '', string.punctuation))
    ans=anss.lower()
    return ans 

df['clean_review']=df['review'].apply(spl)
#print(df.head())

#step2: stopwords

english_stopwords = set(stopwords.words("english"))
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in english_stopwords]
    return " ".join(filtered_words)

df['clean_review'] = df['clean_review'].apply(remove_stopwords)
#print(df.head())

#step3: lemmatization

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [Word(word).lemmatize("v") for word in words]
    return ' '.join(lemmatized_words)

df["clean_review"]=df["clean_review"].apply(lemmatize_text)
print(df.head())

#step 4: stemming

def stem_text(text):
    return porter.stem(text)

df["clean_review"] = df["clean_review"].apply(lambda x: stem_text(x))


#step 5: removing emojis
def remove_emojis(text):
    dem=demoji.findall(text)
    print(dem)
    for item in dem.keys():
        text=text.replace(item , '')
        
    return text  
df["clean_review"] = df["clean_review"].apply(lambda x: remove_emojis(x).replace("''",'')) 
# ans = remove_emojis(text)
# ansa=print(ans.replace("''",'')) 




                                                 url        ASIN gender  \
0  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B0C843K9P9   mens   
1  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B0CQRJVB5F   mens   
2  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B09T45VHWX   mens   
3  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B0BW1YJ38R   mens   
4  https://www.amazon.com/Wrangler-Authentics-Cla...  B08RBHL3HW   mens   

                           manufacturer     category  \
0                     Amazon Essentials  men fashion   
1  Wrangler Authentics Men's Sportswear  men fashion   
2                     Fruit of the Loom  men fashion   
3                                   Lee  men fashion   
4                              Wrangler  men fashion   

                                         description  \
0  SLIM FIT: Slim fit through the hip and thigh w...   
1  RELAXED FIT. These cargo shorts sit at the nat...   
2  Relaxed Fit, 2 Pack Jersey Shorts - 20 pl

In [20]:
df['review'].loc[0]


'I am 5\'7"-5\'8", about 185 lbs. These shorts fit me just right, but if I were to gain 5-10 extra lbs, I would have to go up in waist size.\n\nThese shorts have an excellent "fit." The length is perfect for me. They fit me above the knee, but not too high up where they look like short-shorts. The pockets are deep enough to help keep your items from falling out. Also, I notice they have a YKK brand zipper, which are sturdy, and don\'t de-rail. I don\'t see them much anymore, so I was pleasantly surprised with that quality of craftsmanship. Finally, these shorts have a nice, simple label with the waist size, and another one with the style "e.g., slim" which I find helpful in case I was to re-buy this or a similar product., It’s okay, Nothing fancy, comforting to wear, inexpensive , wearing this for few days inside my home ,Fits well for me\nSo far I like it ., Runs a couple sizes too small. So beware of that. Good looking shorts but make sure to size up if you’ve got that sexy dad bod.,

In [21]:
df['clean_review'].loc[0]

'5758 185 lbs short fit right gain 510 extra lbs would go waist size short excellent fit length perfect fit knee high look like shortshorts pocket deep enough help keep items fall also notice ykk brand zipper sturdy dont derail dont see much anymore pleasantly surprise quality craftsmanship finally short nice simple label waist size another one style eg slim find helpful case rebuy similar product okay nothing fancy comfort wear inexpensive wear days inside home fit well far like run couple size small beware good look short make sure size youve get sexy dad bod good quality fit perfectly order short great theyre right fit dont feel heavy thin feel perfect ive walk comfortable last year buy short 7 inseam great year decide buy 9 inseam sure happen fit buy size 7 tight return buy larger size still fit different tight lose legs wide bigger pair lose weight gain last year short last year 7 still fit fine style material size length different sure happen love 7 inseam many pair many differen

In [22]:
#now fixing reviewer names:


def spl(text):
    cleaned_text = re.sub(r'[^\w\s,]', '', text)  # punctuation + special characters excluding commas
    ans = cleaned_text.lower()
    return ans


df['r_names']=df['reviewer_name'].apply(spl)

#step2: stopwords
english_stopwords = set(stopwords.words("english"))
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in english_stopwords]
    return " ".join(filtered_words)

df['r_names'] = df['r_names'].apply(remove_stopwords)

#step 5: removing emojis
def remove_emojis(text):
    dem=demoji.findall(text)
    print(dem)
    for item in dem.keys():
        text=text.replace(item , '')
        
    return text  
df["r_names"] = df["r_names"].apply(lambda x: remove_emojis(x).replace("''",'')) 




{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{

In [23]:
df['r_names']

0       amazon customer, max, heidi, lucas chau, rober...
1       devin, amazon customer, seth tucker, franklin ...
2       marke, kool aid, jan, sheila, q, lu ann, micha...
3       richard dav, ibuystuffonline, b bad, mmn, bwc,...
4       founding father, john sparks, bob h, amazon cu...
                              ...                        
1428                                               review
1429    karen c rogers, arlena s, cologirl, sy, wender...
1430    cape cod katie, danny penn, kelly, susan mogan...
1431    miss kat, lorraine m, monica zech, kindle cust...
1432    amazon shopper , ncgal, sandi, random shopper,...
Name: r_names, Length: 1220, dtype: object

In [24]:
df['reviewer_name']

0       Amazon Customer, Max, heidi, Lucas Chau, Rober...
1       Devin, Amazon Customer, Seth A. Tucker, Frankl...
2       MarkE, Kool Aid, Jan, Sheila, Q, Lu Ann, Micha...
3       Richard Dav, Ibuystuffonline, B. Bad, MMN, BWC...
4       Founding Father, John Sparks, Bob H, Amazon Cu...
                              ...                        
1428                                            no review
1429    karen c rogers, Arlena S., cologirl, Sy, Wende...
1430    Cape Cod Katie, Danny Penn, Kelly, susan mogan...
1431    Miss KAT, Lorraine M., Monica Zech, Kindle Cus...
1432    Amazon Shopper :), NCGal, Sandi, Random Shoppe...
Name: reviewer_name, Length: 1220, dtype: object

In [25]:
df.head()

,url,ASIN,gender,manufacturer,category,description,subCategory,price,title,brandName,...,product reviews,images,color_imgs,colors,reviewer_name,review_date,review,decs,clean_review,r_names
0,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B0C843K9P9,mens,Amazon Essentials,men fashion,SLIM FIT: Slim fit through the hip and thigh w...,"shorts , tshirts , jackets",12.00,"Amazon Essentials Men's Slim-Fit 9"" Short",Amazon Essentials Store,...,"[{'r_name': 'Amazon Customer', 'date_r': 'Revi...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31B35LtII...,"[Black, Blue, Grey, Grey Pineapple, Grey Plaid...","Amazon Customer, Max, heidi, Lucas Chau, Rober...","December 14, 2023, January 5, 2024, December 3...","I am 5'7""-5'8"", about 185 lbs. These shorts fi...",slim fit slim fit hip thigh taper leg open sit...,5758 185 lbs short fit right gain 510 extra lb...,"amazon customer, max, heidi, lucas chau, rober..."
1,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B0CQRJVB5F,mens,Wrangler Authentics Men's Sportswear,men fashion,RELAXED FIT. These cargo shorts sit at the nat...,"shorts , tshirts , jackets",26.00,Wrangler Authentics Men's Premium Twill Cargo ...,Wrangler Authentics Store,...,"[{'r_name': 'Devin', 'date_r': 'Reviewed in th...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/41kcIogGe...,"[Acorn Twill, Bullfrog, Camel, Forest Green Ca...","Devin, Amazon Customer, Seth A. Tucker, Frankl...","May 29, 2023, February 24, 2024, March 3, 2024...",Comfortable Fit:\nThe cargo shorts feature a r...,relax fit cargo short sit natural waist relax ...,comfortable fit cargo short feature relax fit ...,"devin, amazon customer, seth tucker, franklin ..."
2,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B09T45VHWX,mens,Fruit of the Loom,men fashion,"Relaxed Fit, 2 Pack Jersey Shorts - 20 plus UP...","shorts , tshirts , jackets",19.00,Fruit of the Loom Men's Eversoft Cotton Shorts...,Fruit of the Loom Store,...,"[{'r_name': 'MarkE', 'date_r': 'Reviewed in th...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/21UMhFwFu...,"[2 Pack - Black, 2 Pack - Black Heather, 2 Pac...","MarkE, Kool Aid, Jan, Sheila, Q, Lu Ann, Micha...","February 26, 2024, March 2, 2024, November 12,...","- Fit as expected\n- Comfortable, good for exe...",relax fit 2 pack jersey short 20 plus upf prot...,fit expect comfortable good exercise lounge sl...,"marke, kool aid, jan, sheila, q, lu ann, micha..."
3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,B0BW1YJ38R,mens,Lee,men fashion,STRAIGHT FIT. With a straight fit from the sea...,"shorts , tshirts , jackets",24.00,Lee Men's Extreme Motion Flat Front Short,Lee Store,...,"[{'r_name': 'Richard Dav', 'date_r': 'Reviewed...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31js-qcsQ...,"[Black, Blue Mirage, Charcoal, Clay Rose, Iron...","Richard Dav, Ibuystuffonline, B. Bad, MMN, BWC...","January 11, 2024, October 1, 2023, January 29,...",I love these shorts. I've just bought my 4th p...,straight fit straight fit seat thigh khaki sho...,love short ive buy 4th pair different color lo...,"richard dav, ibuystuffonline, b bad, mmn, bwc,..."
4,https://www.amazon.com/Wrangler-Authentics-Cla...,B08RBHL3HW,mens,Wrangler,men fashion,TRADITIONAL CARGO STYLE. Built for versatility...,"shorts , tshirts , jackets",26.85,Wrangler Authentics Men's Classic Cargo Stretc...,Wrangler Authentics Store,...,"[{'r_name': 'Founding Father', 'date_r': 'Revi...",[https://m.media-amazon.com/images/G/01/HomeCu...,[https://m.media-amazon.com/images/I/31k1p6cNC...,"[Anthracite, Anthracite Camo, Black Twill, Cop...","Founding Father, John Sparks, Bob H, Amazon Cu...","February 27, 2024, February 16, 2024, November...",They run a bit loose. I'm a 34/36 so I ordered...,traditional cargo style build versatility comf...,run bite loose im 3436 order 36 end bite loose...,"founding father, john spar

In [26]:
df.columns

Index(['url', 'ASIN', 'gender', 'manufacturer', 'category', 'description',
       'subCategory', 'price', 'title', 'brandName', 'rating',
       'product reviews', 'images', 'color_imgs', 'colors', 'reviewer_name',
       'review_date', 'review', 'decs', 'clean_review', 'r_names'],
      dtype='object')

In [27]:
columns_to_drop = ['description', 'product reviews', 'reviewer_name','review']
df = df.drop(columns=columns_to_drop)


In [28]:
df.columns

Index(['url', 'ASIN', 'gender', 'manufacturer', 'category', 'subCategory',
       'price', 'title', 'brandName', 'rating', 'images', 'color_imgs',
       'colors', 'review_date', 'decs', 'clean_review', 'r_names'],
      dtype='object')

In [29]:

df['img_id'] = ['ABC' + str(i).zfill(3) for i in range(len(df))]
df['category_id'] = ['XYZ' + str(i).zfill(3) for i in range(len(df))]
df['review_id'] = ['MNO' + str(i).zfill(3) for i in range(len(df))]

print(df)


                                                    url        ASIN  gender  \
0     https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B0C843K9P9    mens   
1     https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B0CQRJVB5F    mens   
2     https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B09T45VHWX    mens   
3     https://www.amazon.com/sspa/click?ie=UTF8&spc=...  B0BW1YJ38R    mens   
4     https://www.amazon.com/Wrangler-Authentics-Cla...  B08RBHL3HW    mens   
...                                                 ...         ...     ...   
1428  https://www.amazon.com/Avenue-Plus-Size-Trouse...  B0CNWRYVHZ  womens   
1429  https://www.amazon.com/Gold-Toe-Womens-Diamond...  B0CV817MB8  womens   
1430  https://www.amazon.com/Berkshire-womens-Trouse...  B0CP8QJGBW  womens   
1431  https://www.amazon.com/Jostar-Womens-Stretchy-...  B00DOHRUE0   women   
1432  https://www.amazon.com/Amazon-Essentials-Women...  B08JDRD3DJ   women   

                              manufacturer       ca

In [30]:
df.columns

Index(['url', 'ASIN', 'gender', 'manufacturer', 'category', 'subCategory',
       'price', 'title', 'brandName', 'rating', 'images', 'color_imgs',
       'colors', 'review_date', 'decs', 'clean_review', 'r_names', 'img_id',
       'category_id', 'review_id'],
      dtype='object')

In [45]:
#savinf clean data to csv
df.to_csv('amazondataset.csv', index=False)

In [31]:
# DATABASE SCHEMA

In [40]:

import pandas as pd
import pymysql

# Connect to the MySQL database
conn = pymysql.connect(host='localhost', user='user', password='password', database='miningamazon')
cursor = conn.cursor()

# Define your dataframes
product_df = df[['url', 'ASIN', 'gender', 'manufacturer', 'rating', 'price', 'title', 'brandName',  'colors', 'decs' , 'category_id' , 'img_id' , 'review_id']]
#['ASIN','title','decs','rating','colors','brandName','price','manufacturer','gender','url','category_id','img_id']
review_df = df[['review_date',  'clean_review', 'r_names', 'review_id']] # Replace ... with your actual review dataframe
category_df = df[['category_id','category', 'subCategory']] # Replace ... with your actual category dataframe
image_df=df[['images', 'color_imgs','img_id']]

In [33]:
for index, row in category_df.iterrows():
    sql = "INSERT INTO cc (category_id, category, subcategory) VALUES (%s, %s, %s)"
    values = (row['category_id'], row['category'], row['subCategory'])
    cursor.execute(sql, values)
conn.commit()
conn.close()    

In [37]:

for index, row in review_df.iterrows():
    sql = "INSERT INTO rr (review_id, reviewer_name, review_date, review_text) VALUES (%s, %s, %s, %s)"
    values = (row['review_id'], row['r_names'], row['review_date'], row['clean_review'])
    cursor.execute(sql, values)
conn.commit()
conn.close()    
   

In [39]:

for index, row in image_df.iterrows():
    sql = "INSERT INTO iii (image_id, imageurl, clrimages) VALUES (%s, %s, %s)"
    values = (
        str(row['img_id']),
        str(row['images']),
        str(row['color_imgs']) 
    )

    cursor.execute(sql, values)
conn.commit()
conn.close()



In [44]:
for index, row in product_df.iterrows():
    sql = "INSERT INTO products (asin, title, descriptions, rating, color, brand_name, price, manufacturer, gender, url, category_id, image_id , review_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s , %s)"
    values = (
        str(row['ASIN']),
        str(row['title']),
        str(row['decs']),
        float(row['rating']),
        str(row['colors']),
        str(row['brandName']),
        float(row['price']),
        str(row['manufacturer']),
        str(row['gender']),
        str(row['url']),
        str(row['category_id']), 
        str(row['img_id'])      ,
        str(row['review_id'])
    )

    cursor.execute(sql, values)
conn.commit()
conn.close()    
